# Problem statement

The insurance approval process, i.e., underwriting is one of the important and time-consuming tasks in an insurance application processing. When you submit your insurance application, the underwriter of the company evaluates it based on the details that you provide using a rule-based or an ML model and decides whether or not to approve your application. You can refer to this link for a better understanding of the underwriting process though it is not pertinent to the case study.

 

An insurance company InsurePlus wants you to help them with finding the optimal number of staff that they need for their insurance application approval process for the calendar year 2021.  In the industry, the number of staffs is considered as a continuous variable. This is also called a Full-Time Equivalent (FTE) of the staff. For example, if a full-time employee (FTE =1) works for 50 hours a week, 10 hours corresponds to 0.2 FTEs. If the pay for 50 hours a week is $5000, then 0.2 FTE who may be a part-time employee will be paid $1000 (5000*0.2). You can read about the concept of a full-time equivalent here. In this case study, you have been provided with the information that follows.

 

Note: Go through each and every point carefully to not miss out on any information.

 

Problem statement & background
The company operates in three states: A, B and C. The state-wise demand for insurance for the year is shown in the table provided below:
                   

 

The company can either handle an application with the staff that they hire or outsource it to a vendor. Assume that there is no capacity limitation to outsourcing.
If they hire staff, he/she can handle 40 insurance applications per month when he/she works 100% of the workdays. However, there are days that he/she will be unavailable to process applications due to training, off days, etc. 
A staff member’s availability (in percentage) to work on processing the insurance applications for each month is shown in the table given below. As mentioned before, with 100% availability, each member can handle 40 applications.
          
        
A special note of practical relevance: In the industry, staff availability is predicted using a time-motion study. But in this case, you have been given fixed numbers for each month in the table above. You can read more about the time-motion study here if you are curious but please note that it is not required to solve the case study.

 

States A and B have a regulatory restriction that the outsourced insurance applications cannot be more than 30% and 40% of the total number of applications for each month, respectively. 
The table given below shows the cost of the staff vs external resources:
                                             

 

The objective is to optimise the total cost for the application approval process by distributing the right number of applications between the FTEs and the vendors while meeting the monthly demand for each state at the same time.

In [29]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math
from pyomo.environ import *

In [30]:
from __future__ import division
from pyomo.opt import SolverFactory

In [31]:
conda install -c conda-forge glpk

Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... done

# All requested packages already installed.



In [32]:
# Reading the data from Excel workbook

InputData = "Staffing+Data.xlsx"
StaffAv = pd.read_excel(InputData, sheet_name="StaffAvailability")
Cost = pd.read_excel(InputData, sheet_name="Cost")
transCost = pd.read_excel(InputData, sheet_name="ServiceRate")
MgDemandDist = pd.read_excel(InputData, sheet_name="DemandData")

In [33]:
StaffAv.head()

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75
3,A,Apr,0.75,0.85,0.80
4,A,May,0.70,0.85,0.78


In [34]:
Cost.head()

,State,Month,AnnualSalary,MonthlySalary,UnitOutSourceCost
0,A,Jan,60000,5000.0,180
1,A,Feb,60000,5000.0,180
2,A,Mar,60000,5000.0,180
3,A,Apr,60000,5000.0,180
4,A,May,60000,5000.0,180


In [35]:
transCost.head()

,MgAppServedPerMonth
0,40


In [36]:
MgDemandDist.head()

,State,Month,Demand
0,A,Jan,5240
1,A,Feb,4878
2,A,Mar,5942
3,A,Apr,2297
4,A,May,1992


## Data pre-processing 

In [37]:
# Create the required Python data structures for indexes and parameters

# Demand Data
Demand = MgDemandDist.set_index(['State','Month'])['Demand'].to_dict()
Demand

{('A', 'Jan'): 5240,
 ('A', 'Feb'): 4878,
 ('A', 'Mar'): 5942,
 ('A', 'Apr'): 2297,
 ('A', 'May'): 1992,
 ('A', 'Jun'): 2275,
 ('A', 'Jul'): 5334,
 ('A', 'Aug'): 3371,
 ('A', 'Sep'): 3759,
 ('A', 'Oct'): 3529,
 ('A', 'Nov'): 4284,
 ('A', 'Dec'): 5183,
 ('B', 'Jan'): 4927,
 ('B', 'Feb'): 2628,
 ('B', 'Mar'): 2974,
 ('B', 'Apr'): 2338,
 ('B', 'May'): 4020,
 ('B', 'Jun'): 3147,
 ('B', 'Jul'): 4271,
 ('B', 'Aug'): 2620,
 ('B', 'Sep'): 4517,
 ('B', 'Oct'): 4155,
 ('B', 'Nov'): 3137,
 ('B', 'Dec'): 4227,
 ('C', 'Jan'): 1162,
 ('C', 'Feb'): 1967,
 ('C', 'Mar'): 1898,
 ('C', 'Apr'): 2261,
 ('C', 'May'): 2030,
 ('C', 'Jun'): 1642,
 ('C', 'Jul'): 2489,
 ('C', 'Aug'): 2496,
 ('C', 'Sep'): 922,
 ('C', 'Oct'): 2421,
 ('C', 'Nov'): 963,
 ('C', 'Dec'): 1998}

In [38]:
Cost.head()

,State,Month,AnnualSalary,MonthlySalary,UnitOutSourceCost
0,A,Jan,60000,5000.0,180
1,A,Feb,60000,5000.0,180
2,A,Mar,60000,5000.0,180
3,A,Apr,60000,5000.0,180
4,A,May,60000,5000.0,180


In [39]:
# FTE Salary from the data

FTE_Salary = Cost.set_index(['State','Month'])['MonthlySalary'].to_dict()

# Unit Outsurcing Cost
UnitOutSourceCost = Cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

In [40]:
FTE_Salary

{('A', 'Jan'): 5000.0,
 ('A', 'Feb'): 5000.0,
 ('A', 'Mar'): 5000.0,
 ('A', 'Apr'): 5000.0,
 ('A', 'May'): 5000.0,
 ('A', 'Jun'): 5000.0,
 ('A', 'Jul'): 5000.0,
 ('A', 'Aug'): 5000.0,
 ('A', 'Sep'): 5000.0,
 ('A', 'Oct'): 5000.0,
 ('A', 'Nov'): 5000.0,
 ('A', 'Dec'): 5000.0,
 ('B', 'Jan'): 4583.333333333333,
 ('B', 'Feb'): 4583.333333333333,
 ('B', 'Mar'): 4583.333333333333,
 ('B', 'Apr'): 4583.333333333333,
 ('B', 'May'): 4583.333333333333,
 ('B', 'Jun'): 4583.333333333333,
 ('B', 'Jul'): 4583.333333333333,
 ('B', 'Aug'): 4583.333333333333,
 ('B', 'Sep'): 4583.333333333333,
 ('B', 'Oct'): 4583.333333333333,
 ('B', 'Nov'): 4583.333333333333,
 ('B', 'Dec'): 4583.333333333333,
 ('C', 'Jan'): 4416.666666666667,
 ('C', 'Feb'): 4416.666666666667,
 ('C', 'Mar'): 4416.666666666667,
 ('C', 'Apr'): 4416.666666666667,
 ('C', 'May'): 4416.666666666667,
 ('C', 'Jun'): 4416.666666666667,
 ('C', 'Jul'): 4416.666666666667,
 ('C', 'Aug'): 4416.666666666667,
 ('C', 'Sep'): 4416.666666666667,
 ('C', 'Oc

In [41]:
UnitOutSourceCost

{('A', 'Jan'): 180,
 ('A', 'Feb'): 180,
 ('A', 'Mar'): 180,
 ('A', 'Apr'): 180,
 ('A', 'May'): 180,
 ('A', 'Jun'): 180,
 ('A', 'Jul'): 180,
 ('A', 'Aug'): 180,
 ('A', 'Sep'): 180,
 ('A', 'Oct'): 180,
 ('A', 'Nov'): 180,
 ('A', 'Dec'): 180,
 ('B', 'Jan'): 150,
 ('B', 'Feb'): 150,
 ('B', 'Mar'): 150,
 ('B', 'Apr'): 150,
 ('B', 'May'): 150,
 ('B', 'Jun'): 150,
 ('B', 'Jul'): 150,
 ('B', 'Aug'): 150,
 ('B', 'Sep'): 150,
 ('B', 'Oct'): 150,
 ('B', 'Nov'): 150,
 ('B', 'Dec'): 150,
 ('C', 'Jan'): 160,
 ('C', 'Feb'): 160,
 ('C', 'Mar'): 160,
 ('C', 'Apr'): 160,
 ('C', 'May'): 160,
 ('C', 'Jun'): 160,
 ('C', 'Jul'): 160,
 ('C', 'Aug'): 160,
 ('C', 'Sep'): 160,
 ('C', 'Oct'): 160,
 ('C', 'Nov'): 160,
 ('C', 'Dec'): 160}

In [42]:
# Number of Applications when Employee working Full Time
FTE_AppServeRate = transCost.iloc[0,0]
FTE_AppServeRate

40

In [43]:
StaffAv.head()

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75
3,A,Apr,0.75,0.85,0.80
4,A,May,0.70,0.85,0.78


In [44]:
# Staff Availability for serving the Insurance Applications
# Worst and Best => StaffAv_LB and StaffAv_UB
StaffAv_LB = StaffAv.set_index(['State','Month'])['LB'].to_dict()
StaffAv_UB = StaffAv.set_index(['State','Month'])['UB'].to_dict()
StaffAvPer = StaffAv.set_index(['State','Month'])['StaffAvPer'].to_dict()

In [45]:
StaffAv_LB

{('A', 'Jan'): 0.7,
 ('A', 'Feb'): 0.65,
 ('A', 'Mar'): 0.7,
 ('A', 'Apr'): 0.75,
 ('A', 'May'): 0.7,
 ('A', 'Jun'): 0.65,
 ('A', 'Jul'): 0.6,
 ('A', 'Aug'): 0.65,
 ('A', 'Sep'): 0.7,
 ('A', 'Oct'): 0.65,
 ('A', 'Nov'): 0.6,
 ('A', 'Dec'): 0.6,
 ('B', 'Jan'): 0.7,
 ('B', 'Feb'): 0.65,
 ('B', 'Mar'): 0.7,
 ('B', 'Apr'): 0.75,
 ('B', 'May'): 0.7,
 ('B', 'Jun'): 0.65,
 ('B', 'Jul'): 0.6,
 ('B', 'Aug'): 0.65,
 ('B', 'Sep'): 0.7,
 ('B', 'Oct'): 0.65,
 ('B', 'Nov'): 0.6,
 ('B', 'Dec'): 0.6,
 ('C', 'Jan'): 0.7,
 ('C', 'Feb'): 0.65,
 ('C', 'Mar'): 0.7,
 ('C', 'Apr'): 0.75,
 ('C', 'May'): 0.7,
 ('C', 'Jun'): 0.65,
 ('C', 'Jul'): 0.6,
 ('C', 'Aug'): 0.65,
 ('C', 'Sep'): 0.7,
 ('C', 'Oct'): 0.65,
 ('C', 'Nov'): 0.6,
 ('C', 'Dec'): 0.6}

In [46]:
StaffAv_UB

{('A', 'Jan'): 0.9,
 ('A', 'Feb'): 0.85,
 ('A', 'Mar'): 0.8,
 ('A', 'Apr'): 0.85,
 ('A', 'May'): 0.85,
 ('A', 'Jun'): 0.8,
 ('A', 'Jul'): 0.75,
 ('A', 'Aug'): 0.85,
 ('A', 'Sep'): 0.9,
 ('A', 'Oct'): 0.8,
 ('A', 'Nov'): 0.75,
 ('A', 'Dec'): 0.7,
 ('B', 'Jan'): 0.9,
 ('B', 'Feb'): 0.85,
 ('B', 'Mar'): 0.8,
 ('B', 'Apr'): 0.85,
 ('B', 'May'): 0.85,
 ('B', 'Jun'): 0.8,
 ('B', 'Jul'): 0.75,
 ('B', 'Aug'): 0.85,
 ('B', 'Sep'): 0.9,
 ('B', 'Oct'): 0.8,
 ('B', 'Nov'): 0.75,
 ('B', 'Dec'): 0.7,
 ('C', 'Jan'): 0.9,
 ('C', 'Feb'): 0.85,
 ('C', 'Mar'): 0.8,
 ('C', 'Apr'): 0.85,
 ('C', 'May'): 0.85,
 ('C', 'Jun'): 0.8,
 ('C', 'Jul'): 0.75,
 ('C', 'Aug'): 0.85,
 ('C', 'Sep'): 0.9,
 ('C', 'Oct'): 0.8,
 ('C', 'Nov'): 0.75,
 ('C', 'Dec'): 0.7}

In [47]:
StaffAvPer

{('A', 'Jan'): 0.81,
 ('A', 'Feb'): 0.76,
 ('A', 'Mar'): 0.75,
 ('A', 'Apr'): 0.8,
 ('A', 'May'): 0.78,
 ('A', 'Jun'): 0.73,
 ('A', 'Jul'): 0.68,
 ('A', 'Aug'): 0.76,
 ('A', 'Sep'): 0.81,
 ('A', 'Oct'): 0.73,
 ('A', 'Nov'): 0.68,
 ('A', 'Dec'): 0.65,
 ('B', 'Jan'): 0.81,
 ('B', 'Feb'): 0.76,
 ('B', 'Mar'): 0.75,
 ('B', 'Apr'): 0.8,
 ('B', 'May'): 0.78,
 ('B', 'Jun'): 0.73,
 ('B', 'Jul'): 0.68,
 ('B', 'Aug'): 0.76,
 ('B', 'Sep'): 0.81,
 ('B', 'Oct'): 0.73,
 ('B', 'Nov'): 0.68,
 ('B', 'Dec'): 0.65,
 ('C', 'Jan'): 0.81,
 ('C', 'Feb'): 0.76,
 ('C', 'Mar'): 0.75,
 ('C', 'Apr'): 0.8,
 ('C', 'May'): 0.78,
 ('C', 'Jun'): 0.73,
 ('C', 'Jul'): 0.68,
 ('C', 'Aug'): 0.76,
 ('C', 'Sep'): 0.81,
 ('C', 'Oct'): 0.73,
 ('C', 'Nov'): 0.68,
 ('C', 'Dec'): 0.65}

In [48]:
Cost['State'].unique()

array(['A', 'B', 'C'], dtype=object)

In [49]:
Cost['Month'].unique()

array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'], dtype=object)

In [50]:
# Index

BankLoc = Cost['State'].unique()

Month = Cost['Month'].unique()

In [51]:
BankLoc.tolist()

['A', 'B', 'C']

# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



##### InsurePlus wants to optimise the cost for staffing plan so we derive following mathematical model

#### <b> Data given:<b>
1) Locations:  A, B, C<br>
2) Demand data provided for statewise and monthwise<br>
3) Resources: Full Time Employees, Outsourced Application<br>
4) Staff Availablity statewise and monthwise<br>
5) Staff Annual Salary statewise astatewise and monthwise<br>
6) Outsource Cost per application statewise and monthwise<br>
7) Outsourcing restriction at state A & B @ 30% & 40%  respectively  


### Mathematical Model:

#### <b>Index:<b> 
$i = State$<br>
$j = Month$

#### <b>Parameters:<b> 
Demand = $Demand(i,j)$<br> 
Staff Availability Percentage = $StaffAv(i,j)$ <br>
FTESalary = $FTESalary(i,j)$<br>
Unit Outsource Cost = $UnitCost(i,j)$ <br>
FTEAppServeRate = $(No. of Appl / Month)$ 
            
#### <b>Decision Variables:<b>
$FTE Equivallent at any location, month (Continuous Var) = X(i,j) $<br>
$OutSource App Qty in a month (Integer) = Y(i,j) $
                    
#### <b>Objective Function:<b>
$Minimize Cost = Annual Total Cost of FTE + Annual Total Cost of Outsourced Application$ <br>
    
$\sum_{i }\sum_{j }FTESalary(i,j)*X(i,j) + \sum_{i}\sum_{j}UnitCost(i,j)*Y(i,j) \ >= C_{min}$<br>



**Note:** You can write the mathematical formulation directly in your report.

# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [56]:
# Creating a model instance
# model object

model = ConcreteModel()

In [57]:
# Define Pyomo sets and Parameters

# Next task is to define the Sets
# i Insurance Company location
# j Months

model.i = Set(initialize = BankLoc.tolist(), doc='State')
model.j = Set(initialize = Month.tolist(), doc='Month')

# Parameters -> Demand, FTE salaries, Outsourcing cost, Average Staff Availability

model.d = Param(model.i, model.j, initialize=Demand, doc='Demand')
model.s = Param(model.i, model.j, initialize=FTE_Salary, doc='FTE_Salary')
model.oc = Param(model.i, model.j, initialize=UnitOutSourceCost, doc='UnitSourceCost')
model.sa = Param(model.i, model.j, initialize=StaffAvPer, doc='StaffAvPercent')

# Scalar Application Serve Rate when working 100%

model.r = Param(initialize=FTE_AppServeRate, doc='FTE App Serve Rate')

# Parameter for no of Applications that can be processed for the given staff availability

def c_FTEAPP(model, i, j):
    return model.r * model.sa[i,j]

model.FTEAPPNO = Param(model.i, model.j, initialize=c_FTEAPP, doc='NO OF FTE APP')

# Regulatory Parameters with Regulatory Constraint
# A -> 30%
def Reg_A(model, i, j):
    return 0.30 * model.d[i,j]

model.Reg_A = Param(model.i, model.j, initialize=Reg_A, doc='RegRest_A')

# for B
# B -> 40%
def Reg_B(model, i, j):
    return 0.40 * model.d[i,j]

model.Reg_B = Param(model.i, model.j, initialize=Reg_B, doc='RegRest_B')

In [58]:
# Decision variables

model.x = Var(model.i, model.j, domain=NonNegativeReals, doc='No of FTE')

model.y = Var(model.i, model.j, domain=NonNegativeIntegers, doc='No of Outsourced App')

In [59]:
# Constraints

# Demand Constraints

model.demand_constraint = ConstraintList()
for i in model.i:
    for j in model.j:
        model.demand_constraint.add(expr= model.x[i,j]*model.FTEAPPNO[i,j] + model.y[i,j] == model.d[i,j])

# Regulatory Constraint

model.regulatoryCons = ConstraintList()
for i in model.i:
    for j in model.j:
        if i=='A':
            model.regulatoryCons.add(expr= model.y['A',j] <= model.Reg_A['A',j])
        elif i=='B':
            model.regulatoryCons.add(expr= model.y['B',j] <= model.Reg_B['B',j])    

In [60]:
# Objective Function

def objective_rule(model):
    return sum(model.s[i,j]*model.x[i,j] for i in model.i for j in model.j) +\
           sum(model.oc[i,j]*model.y[i,j] for i in model.i for j in model.j)

model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')

In [61]:
# Invoking the solver

result_cost = SolverFactory('glpk').solve(model)
result_cost.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17962336.4487699
  Upper bound: 17962336.4487699
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.20194578170776367
# ----------------------------------------------------------
#   Solution Information
# ------------

In [62]:
# Print the value of the objective function

TotalCost = model.objective.expr()
TotalCost

17962336.448769882

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [63]:
# Creating dataframe for the results

FTE_Staff = []
for i in BankLoc:
    for j in Month:
        v = model.x[i,j].value
        w = model.y[i,j].value
        FTE_Staff.append([i,j,v,w]) # list of list
        
FTE_Staff = pd.DataFrame(FTE_Staff, columns=['State','Month','FTE','App_Outsourced'])
FTE_Staff.head()

,State,Month,FTE,App_Outsourced
0,A,Jan,161.728395,0.0
1,A,Feb,160.460526,0.0
2,A,Mar,198.066667,0.0
3,A,Apr,71.781250,0.0
4,A,May,63.846154,0.0


In [64]:
# Writing the results in to an Excel sheet



# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [120]:
# Creating a model instance

# model object

modelwc = ConcreteModel()

In [121]:
# Define Pyomo sets and Parameters

# Next task is to define the Sets
# i Insurance Company location
# j Months

modelwc.i = Set(initialize = BankLoc.tolist(), doc='State')
modelwc.j = Set(initialize = Month.tolist(), doc='Month')

# Parameters -> Demand, FTE salaries, Outsourcing cost, Average Staff Availability

modelwc.d = Param(modelwc.i, modelwc.j, initialize=Demand, doc='Demand')
modelwc.s = Param(modelwc.i, modelwc.j, initialize=FTE_Salary, doc='FTE_Salary')
modelwc.oc = Param(modelwc.i, modelwc.j, initialize=UnitOutSourceCost, doc='UnitSourceCost')
modelwc.sa = Param(modelwc.i, modelwc.j, initialize=StaffAv_LB, doc='StaffAvLB')

# Scalar Application Serve Rate when working 100%

modelwc.r = Param(initialize=FTE_AppServeRate, doc='FTE App Serve Rate')

def c_FTEAPP(modelwc, i, j):
    return modelwc.r * modelwc.sa[i,j]

modelwc.FTEAPPNO = Param(modelwc.i, modelwc.j, initialize=c_FTEAPP, doc='NO OF FTE APP')

# Regulatory Parameters with Regulatory Constraint
# A -> 30%
def Reg_A(modelwc, i, j):
    return 0.30 * modelwc.d[i,j]

modelwc.Reg_A = Param(modelwc.i, modelwc.j, initialize=Reg_A, doc='RegRest_A')

# for B
# B -> 40%
def Reg_B(modelwc, i, j):
    return 0.40 * modelwc.d[i,j]

modelwc.Reg_B = Param(modelwc.i, modelwc.j, initialize=Reg_B, doc='RegRest_B')


In [122]:
# Decision variables

modelwc.x = Var(modelwc.i, modelwc.j, domain=NonNegativeReals, doc='No of FTE')

modelwc.y = Var(modelwc.i, modelwc.j, domain=NonNegativeIntegers, doc='No of Outsourced App')

In [123]:
# Constraints

# Demand Constraints

modelwc.demand_constraint = ConstraintList()
for i in modelwc.i:
    for j in modelwc.j:
        modelwc.demand_constraint.add(expr= modelwc.x[i,j]*modelwc.FTEAPPNO[i,j] + modelwc.y[i,j] == modelwc.d[i,j])

# Regulatory Constraint

modelwc.regulatoryCons = ConstraintList()
for i in modelwc.i:
    for j in modelwc.j:
        if i=='A':
            modelwc.regulatoryCons.add(expr= modelwc.y['A',j] <= modelwc.Reg_A['A',j])
        elif i=='B':
            modelwc.regulatoryCons.add(expr= modelwc.y['B',j] <= modelwc.Reg_B['B',j])    

In [124]:
# Objective function

def objective_rule(modelwc):
    return sum(modelwc.s[i,j]*modelwc.x[i,j] for i in modelwc.i for j in modelwc.j) +\
           sum(modelwc.oc[i,j]*modelwc.y[i,j] for i in modelwc.i for j in modelwc.j)

modelwc.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')

In [125]:
# Invoking the solver
result_cost_wc = SolverFactory('glpk').solve(modelwc)
result_cost_wc.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 19599482.5167888
  Upper bound: 19599482.5167888
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.12647294998168945
# ----------------------------------------------------------
#   Solution Information
# ------------

In [126]:
# Print the value of the objective function

TotalCostWC = modelwc.objective.expr()
TotalCostWC

19599482.516788766

**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [127]:
# Creating dataframe for the results

FTE_StaffWC = []
for i in BankLoc:
    for j in Month:
        v = modelwc.x[i,j].value
        w = modelwc.y[i,j].value
        FTE_StaffWC.append([i,j,v,w]) # list of list
        
FTE_StaffWC = pd.DataFrame(FTE_StaffWC, columns=['State','Month','FTE','App_Outsourced'])
FTE_StaffWC.head()

,State,Month,FTE,App_Outsourced
0,A,Jan,187.142857,0.0
1,A,Feb,131.346154,1463.0
2,A,Mar,212.214286,0.0
3,A,Apr,76.566667,0.0
4,A,May,71.142857,0.0


#### 3.1.2 Percentage of outsourced applications 

In [128]:
# write your code here

TotlAppDemand = MgDemandDist['Demand'].sum() 
TotlAppDemand

PercentageOutsourced = ((FTE_StaffWC['App_Outsourced'].sum() / TotlAppDemand)* 100).round(2)
PercentageOutsourced

35.14

#### 3.1.3 Average cost per application

In [129]:
# write your code here

TotalFTEWC = FTE_StaffWC['FTE'].sum()
AverageFTEPerMonthWC = (TotalFTEWC/12).round(2)
print(AverageFTEPerMonthWC) 

AverageCostPerApplWC = (TotalCostWC / TotlAppDemand).round(2)
print(AverageCostPerApplWC)

230.11
173.0


### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members


In [130]:
# Creating a model instance

# model object

modelbc = ConcreteModel()

In [131]:
# Define Pyomo sets and Parameters

# Next task is to define the Sets
# i Insurance Company location
# j Months

modelbc.i = Set(initialize = BankLoc.tolist(), doc='State')
modelbc.j = Set(initialize = Month.tolist(), doc='Month')

# Parameters -> Demand, FTE salaries, Outsourcing cost, Average Staff Availability

modelbc.d = Param(modelbc.i, modelbc.j, initialize=Demand, doc='Demand')
modelbc.s = Param(modelbc.i, modelbc.j, initialize=FTE_Salary, doc='FTE_Salary')
modelbc.oc = Param(modelbc.i, modelbc.j, initialize=UnitOutSourceCost, doc='UnitSourceCost')
modelbc.sa = Param(modelbc.i, modelbc.j, initialize=StaffAv_UB, doc='StaffAvUB')

# Scalar Application Serve Rate when working 100%

modelbc.r = Param(initialize=FTE_AppServeRate, doc='FTE App Serve Rate')

def c_FTEAPP(modelbc, i, j):
    return modelbc.r * modelbc.sa[i,j]

modelbc.FTEAPPNO = Param(modelbc.i, modelbc.j, initialize=c_FTEAPP, doc='NO OF FTE APP')

# Regulatory Parameters with Regulatory Constraint
# A -> 30%
def Reg_A(modelbc, i, j):
    return 0.30 * modelbc.d[i,j]

modelbc.Reg_A = Param(modelbc.i, modelbc.j, initialize=Reg_A, doc='RegRest_A')

# for B
# B -> 40%
def Reg_B(modelbc, i, j):
    return 0.40 * modelbc.d[i,j]

modelbc.Reg_B = Param(modelbc.i, modelbc.j, initialize=Reg_B, doc='RegRest_B')


In [132]:
# Decision variables

modelbc.x = Var(modelbc.i, modelbc.j, domain=NonNegativeReals, doc='No of FTE')

modelbc.y = Var(modelbc.i, modelbc.j, domain=NonNegativeIntegers, doc='No of Outsourced App')

In [133]:
# Constraints

# Demand Constraints

modelbc.demand_constraint = ConstraintList()
for i in modelbc.i:
    for j in modelbc.j:
        modelbc.demand_constraint.add(expr= modelbc.x[i,j]*modelbc.FTEAPPNO[i,j] + modelbc.y[i,j] == modelbc.d[i,j])

# Regulatory Constraint

modelbc.regulatoryCons = ConstraintList()
for i in modelbc.i:
    for j in modelbc.j:
        if i=='A':
            modelbc.regulatoryCons.add(expr= modelbc.y['A',j] <= modelbc.Reg_A['A',j])
        elif i=='B':
            modelbc.regulatoryCons.add(expr= modelbc.y['B',j] <= modelbc.Reg_B['B',j])    

In [134]:
# Objective function

def objective_rule(modelbc):
    return sum(modelbc.s[i,j]*modelbc.x[i,j] for i in modelbc.i for j in modelbc.j) +\
           sum(modelbc.oc[i,j]*modelbc.y[i,j] for i in modelbc.i for j in modelbc.j)

modelbc.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')

In [135]:
# Invoking the solver
result_cost_bc = SolverFactory('glpk').solve(modelbc)
result_cost_bc.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16527535.6379357
  Upper bound: 16527535.6379357
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.4506673812866211
# ----------------------------------------------------------
#   Solution Information
# -------------

In [136]:
# Print the value of the objective function

TotalCostBC = modelbc.objective.expr()
TotalCostBC

16527535.63793573

**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [137]:
# Creating dataframe for the results
FTE_StaffBC = []
for i in BankLoc:
    for j in Month:
        v = modelbc.x[i,j].value
        w = modelbc.y[i,j].value
        FTE_StaffBC.append([i,j,v,w]) # list of list
        
FTE_StaffBC = pd.DataFrame(FTE_StaffBC, columns=['State','Month','FTE','App_Outsourced'])
FTE_StaffBC.head()

,State,Month,FTE,App_Outsourced
0,A,Jan,145.555556,0.0
1,A,Feb,143.470588,0.0
2,A,Mar,185.687500,0.0
3,A,Apr,67.558824,0.0
4,A,May,58.588235,0.0


#### 3.2.2 Percentage of outsourced applications

In [138]:
# write your code here

TotlAppDemand = MgDemandDist['Demand'].sum() 
TotlAppDemand

PercentageOutsourcedBC = ((FTE_StaffBC['App_Outsourced'].sum() / TotlAppDemand)* 100).round(2)
PercentageOutsourcedBC

4.11

#### 3.2.3 Average cost per application

In [139]:
# write your code here

TotalFTEBC = FTE_StaffBC['FTE'].sum()
AverageFTEPerMonthBC = (TotalFTEBC/12).round(2)
print(AverageFTEPerMonthBC) 

AverageCostPerApplBC = (TotalCostBC / TotlAppDemand).round(2)
print(AverageCostPerApplBC)

278.6
145.88


# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.